In [1298]:
import numpy as np
from abc import ABCMeta, abstractmethod
from sklearn import datasets

sigmoid = np.vectorize(lambda x: 1 / (1 + np.exp(-x)))
dsigmoid = np.vectorize(lambda x: x*(1-x))
relu = np.vectorize(lambda x: max(0,x))
dcost = np.vectorize(lambda y,a: -y/a +((1-y)/(1-a)))
prob_to_class = np.vectorize(lambda x: 0 if (x < 0.5) else 1)

In [1755]:
class LinearLayer :
    def __init__(self,in_features, out_features):
        self.weights = np.random.randn(out_features, in_features) * 0.01
        self.bias = np.zeros((out_features,1))
        
    def forward(self,X):
        self.X = X
        return np.dot(self.weights,X) + self.bias
    
    def backward(self, grad):
        m = X.shape[1]
        self.grad_w = (1/m)*np.dot(grad, self.X.T)
        self.grad_b = (1/m)*np.sum(grad, axis=1, keepdims=True)
        d = np.dot(self.weights.T,grad)
        return d
    
    def updateParameters(self, learningRate):
        self.weights = self.weights - learningRate*self.grad_w
        self.bias = self.bias - learningRate*self.grad_b


In [1756]:
class SigmoidLayer:
    
    def forward(self,Z):
        self.Z = Z
        self.activation = sigmoid(Z)
        return self.activation
    
    def backward(self, grad):
        sigmoidGrad = dsigmoid(self.activation) * grad
        return sigmoidGrad
    
    def updateParameters(self, learingRate):
        return
        

In [1757]:
class CostLayer:
    
    def forward(self, AL, Y):
        self.AL = AL
        self.Y = Y
        self.numExamples = Y.shape[1]
        cost = (-1/self.numExamples)*(np.dot(np.log(AL),self.Y.T) + np.dot(np.log(1-AL), (1-self.Y.T)))
        cost = np.squeeze(cost)
        return cost
    
    def backward(self):
        ones = -(self.Y * self.AL)
        zeros = (1-self.Y) * (1-self.AL)
        costGrad = 1 / (zeros  + ones)
        return costGrad

In [1758]:
class LogisticRegressionNetwork:
    def __init__(self, in_features):
        self.in_features = in_features
        self.reset()
        
    def reset(self):
        self.linearLayer = LinearLayer(self.in_features, 1)
        self.sigmoidLayer = SigmoidLayer()
        self.costLayer = CostLayer()
        
    def forward(self,X,Y):
        f1 = self.linearLayer.forward(X)
        f2 = self.sigmoidLayer.forward(f1)
        cost = self.costLayer.forward(f2,Y)
        return cost
    
    def backward(self):
        b1 = self.costLayer.backward()
        b2 = self.sigmoidLayer.backward(b1)
        b3 = self.linearLayer.backward(b2)
    
    def updateParameters(self, learningRate):
        self.linearLayer.updateParameters(learningRate)
        
    def train(self, X,Y, epochs = 10, learningRate = 0.1):
        self.reset()
        for epoch in range(0,epochs):
            cost = self.forward(X,Y)
            self.backward()
            self.updateParameters(learningRate)
            print(cost)
            
    def predict(self,X,Y):
        self.forward(X,Y)
        return prob_to_class(self.sigmoidLayer.activation)

In [1759]:
class FeedForwardNeuralNetwork:
    def __init__(self, layer_dims):
        self.layers = []
        self.layer_dims = layer_dims
        dim_in = layer_dims[0]
        for i in range(1, len(layer_dims)):
            dim_out = layer_dims[i]
            self.layers.append(LinearLayer(dim_in, dim_out))
            self.layers.append(SigmoidLayer())
            dim_in = dim_out
        self.costLayer = CostLayer()
        
    def forward(self,X,Y):
        A = X
        for i in range(0,len(self.layers)):
            A = self.layers[i].forward(A)
        cost = self.costLayer.forward(A,Y)
        return cost
    
    def backward(self):
        derivatives = self.costLayer.backward()
        for i in reversed(range(0,len(self.layers))):
            derivatives = self.layers[i].backward(derivatives)
            
    def updateParameters(self, learningRate):
        for i in range(0,len(self.layers)):
            self.layers[i].updateParameters(learningRate)
            
    def train(self, X,Y, epochs = 10, learningRate = 0.01):
        self.__init__(self.layer_dims)
        for epoch in range(0,epochs):
            cost = self.forward(X,Y)
            self.backward()
            self.updateParameters(learningRate)
            print(cost)
            
    def predict(self,X,Y):
        self.forward(X,Y)
        print("activations = ", self.layers[len(self.layers)-1].activation)
        return prob_to_class(self.layers[len(self.layers)-1].activation)
        
    

In [1760]:
X, y = datasets.make_classification(n_samples=30, n_features=4)

In [1761]:
y = np.reshape(a=y,newshape=(1,30))
X = X.T

In [1762]:
print("X shape = ", X.shape)
print("y shape = ", y.shape)

X shape =  (4, 30)
y shape =  (1, 30)


In [1763]:
lr_model = LogisticRegressionNetwork(4)

In [1764]:
lr_model.train(X,y,epochs=100,learningRate=0.1)

0.690912904871607
0.6587353050207142
0.6305816022226955
0.6058920689308633
0.5841687254688104
0.5649790025045318
0.5479531420254075
0.5327782542007429
0.519191064277646
0.5069705785191434
0.4959313110770016
0.4859173403954225
0.4767972509656059
0.4684599082522008
0.4608109687463329
0.45377001513113163
0.4472682113830705
0.4412463846158212
0.43565345449079773
0.430445144579345
0.425582922109686
0.42103312275508104
0.4167662255637924
0.41275624999301463
0.40898025252984477
0.40541790479946005
0.4020511385845793
0.39886384598904645
0.3958416252181472
0.3929715642368472
0.390242055998438
0.38764264008442323
0.38516386652055085
0.3827971782798218
0.38053480958747066
0.37836969763390826
0.3762954057020391
0.37430605604306444
0.37239627110398543
0.3705611219317807
0.3687960827625922
0.3670969909563644
0.36546001156400115
0.36388160591981294
0.36235850374058837
0.3608876782870183
0.359466324205894
0.35809183772449077
0.35676179891344645
0.35547395577261165
0.3542262099268643
0.3530166037466626

In [1765]:
lr_model.predict(X,y)

array([[1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 1, 1, 0, 1]])

In [1766]:
y

array([[1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 0, 1, 0, 1, 0, 1]])

In [1802]:
NN_model = FeedForwardNeuralNetwork([4,10,1])

In [1804]:
NN_model.train(X,y,epochs=1000,learningRate=1)

0.6931541186693346
0.6931091264590774
0.6930701651284168
0.6930265865628703
0.6929748851056797
0.6929111143606831
0.6928304991300378
0.6927270782541999
0.6925932697624846
0.6924193282062504
0.6921926599523018
0.6918969564105621
0.6915110994036752
0.6910077887339692
0.6903518423160565
0.6894981289172679
0.6883891205167285
0.6869521075843388
0.6850962228747188
0.6827095877096954
0.6796571470678223
0.6757800979992995
0.6708982000354401
0.6648165679116627
0.6573385570927177
0.6482857359164098
0.6375243950914891
0.6249955652014596
0.610742685155955
0.5949291611041541
0.577838573202772
0.5598539777458169
0.5414187427694828
0.5229872154998821
0.5049765278361189
0.4877296716477056
0.4714954838023847
0.456425728783212
0.4425853476832109
0.42997021931731244
0.4185271751578327
0.40817255557218496
0.39880734377095123
0.39032830378185823
0.3826354098616861
0.37563625539301915
0.3692482212058871
0.3633991093231703
0.358026808792975
0.35307841321018446
0.3485090814656398
0.3442808325174382
0.34036139

0.2612335982668663
0.2611870862175701
0.2611405915419218
0.2610941141772035
0.26104765405999064
0.2610012111261544
0.26095478531086397
0.2609083765485882
0.26086198477309763
0.2608156099174665
0.26076925191407435
0.26072291069460807
0.2606765861900631
0.2606302783307457
0.2605839870462736
0.26053771226557837
0.26049145391690653
0.26044521192782066
0.260398986225201
0.26035277673524687
0.2603065833834773
0.26026040609473283
0.2602142447931761
0.260168099402293
0.26012196984489383
0.2600758560431142
0.26002975791841537
0.25998367539158596
0.25993760838274177
0.25989155681132736
0.25984552059611593
0.2597994996552107
0.25975349390604463
0.2597075032653815
0.25966152764931644
0.25961556697327587
0.25956962115201826
0.2595236900996346
0.2594777737295477
0.2594318719545141
0.2593859846866226
0.25934011183729555
0.2592942533172883
0.2592484090366898
0.25920257890492227
0.2591567628307411
0.2591109607222355
0.2590651724868276
0.2590193980312728
0.2589736372616595
0.2589278900834091
0.258882156

In [1805]:
NN_model.predict(X,y)

activations =  [[9.70737285e-01 6.64920500e-01 4.74635979e-04 2.22350807e-04
  7.08413158e-01 1.12423345e-01 5.08575576e-04 2.48619083e-04
  3.21337940e-01 9.96106343e-01 9.93841965e-01 8.86780307e-01
  4.60198907e-01 7.14983255e-04 3.97411710e-04 8.91173845e-01
  3.74436898e-04 9.54136770e-01 9.00435560e-01 9.48249500e-01
  9.84555880e-01 3.15302662e-01 9.73979315e-04 8.71902694e-01
  6.31424158e-01 4.06594051e-01 8.88159684e-02 9.03742576e-01
  3.05197748e-04 9.77306602e-01]]


array([[1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 1, 0, 1]])

In [1806]:
y

array([[1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 0, 1, 0, 1, 0, 1]])

In [1693]:
0

0